### Definitions of $ G $ and $ U $

#### Game $ G $
The game $ G = (N, A, U) $ is defined by the following components:

- **$ N = \{1, 2, 3, 4\} $**: The set of cars moving from $ s $ to $ t $.
- **$ R = \{e1, e2, e3, e4, e5\} $**: The set of roads.
- **$ P = \{p1=\{e1, e3\}, p2=\{e2, e4\}, p3=\{e1, e4, e5\}\} $**: The set of paths from the starting point $ s $ to the end point $ t $.
- **$ A_i = P $**: The set of paths that car $ i $ can take. $ A = \prod_{i \in N} A_i $.

#### Cost Function $ c_r(|\{a\}_r|) $
The time it takes to traverse road $ r \in R $ depends on the number of cars $ |\{a\}_r| $ using that road:
- $ c_{e1}(|\{a\}_{e1}|) = 3 + 3 |\{a\}_{e1}| $
- $ c_{e2}(|\{a\}_{e2}|) = 19 + |\{a\}_{e2}| $
- $ c_{e3}(|\{a\}_{e3}|) = 19 + |\{a\}_{e3}| $
- $ c_{e4}(|\{a\}_{e4}|) = 3 + 3 |\{a\}_{e4}| $
- $ c_{e5}(|\{a\}_{e5}|) = 6 + |\{a\}_{e5}| $

#### Objective Function $ W_r(a) $
The objective function for road $ r \in R $ is given by:
$
W_r(a) =
\begin{cases}
|\{a\}_r| \{15 - c_r(|\{a\}_r|)\} & \text{if } r \in \{e1, e4, e5\} \\
|\{a\}_r| \{30 - c_r(|\{a\}_r|)\} & \text{if } r \in \{e2, e3\}
\end{cases}
$

#### Utility Function $ U_i(a) $
The utility function for agent $ i $ (minimizing travel time) is defined as:
$
U_i(a) = \sum_{r \in a_i} \frac{W_r(a)}{|\{a\}_{r}|} = 45 - \sum_{r \in a_i} c_r(|\{a\}_r|)
$

#### Global Objective Function $ W(a) $
The global objective function (minimizing total travel time) is:
$
W(a) = \sum_{r \in R} W_r(a) = \sum_{i \in N} U_i(a)
$

### Potential Function $ \varphi(a) $
To show that this game is a potential game, we define the potential function as:
$
\varphi(a) = -\sum_{r \in R} \sum_{k=1}^{|\{a\}_r|} c_r(k)
$


In [6]:
import numpy as np
import pandas as pd
from itertools import product


# コスト関数を定義します。
# c_e1(k): 道路e1のコスト関数。通行車両数kに応じてコストが増加します。
def c_e1(k):
    return 3 + 3 * k


# c_e2(k): 道路e2のコスト関数。通行車両数kに応じてコストが増加します。
def c_e2(k):
    return 19 + k


# c_e3(k): 道路e3のコスト関数。通行車両数kに応じてコストが増加します。
def c_e3(k):
    return 19 + k


# c_e4(k): 道路e4のコスト関数。通行車両数kに応じてコストが増加します。
def c_e4(k):
    return 3 + 3 * k


# c_e5(k): 道路e5のコスト関数。通行車両数kに応じてコストが増加します。
def c_e5(k):
    return 6 + k


# 目的関数を定義します。
def W_r(r, k):
    # W_r(r, k): 道路rの目的関数。特定の道路rにおける車両数kの時の評価値を計算します。
    # 'e1', 'e4', 'e5'の場合と'e2', 'e3'の場合で異なる計算を行います。
    if r in ['e1', 'e4', 'e5']:
        return k * (15 - eval(f"c_{r}({k})"))
    elif r in ['e2', 'e3']:
        return k * (30 - eval(f"c_{r}({k})"))


# 各車両の効用関数を定義します。
def U_i(a_i, a):
    # U_i(a_i, a): 車両iの効用関数。車両iの選択した経路a_iと全車両の経路aに基づいて効用を計算します。
    # 具体的には、特定の車両が選んだ経路に含まれる各道路のコストの合計を引くことで効用を計算します。
    roads = set(a_i)
    return 45 - sum(eval(f"c_{r}({a.count(r)})") for r in roads)


# グローバル目的関数を定義します。
def W(a):
    # W(a): 全体の目的関数。全車両の経路aに基づいて全体の評価値を計算します。
    # すべての道路について、その道路を通る車両数に対する目的関数の合計を求めます。
    return sum(W_r(r, a.count(r)) for r in set(a))


# ポテンシャル関数を定義します。
def phi(a):
    # phi(a): ポテンシャル関数。全車両の経路aに基づいてポテンシャル値を計算します。
    # 具体的には、すべての道路について、その道路を通る車両数に応じたコストの合計を負の値として計算します。
    return -sum(sum(eval(f"c_{r}({k})") for k in range(1, a.count(r) + 1)) for r in set(a))


# 経路を定義します。
# paths: 経路の辞書。各経路p1, p2, p3は特定の道路のリストとして定義されています。
paths = {
    'p1': ['e1', 'e3'],
    'p2': ['e2', 'e4'],
    'p3': ['e1', 'e4', 'e5']
}

# すべての可能な行動プロファイルを作成します。
action_profiles = list(product(paths.keys(), repeat=4))
# action_profiles: すべての可能な行動プロファイルのリスト。4つの車両がそれぞれp1, p2, p3のいずれかの経路を選択する全組み合わせを生成します。

# ナッシュ均衡とWを最大化するプロファイルを探します。
nash_equilibria = []
max_W = -np.inf
optimal_profile = None

# すべての行動プロファイルについてループします。
for profile in action_profiles:
    # 現在の行動プロファイルに対する各車両の効用を計算します。
    current_U = [U_i(paths[profile[i]], [road for p in profile for road in paths[p]]) for i in range(4)]
    # 各行動プロファイルについてナッシュ均衡をチェックします
    is_nash_equilibrium = True
    for i in range(4):  # 各車両iについて
        # 車両iが現在の経路profile[i]を選択した場合の効用を計算します
        current_utility = U_i(paths[profile[i]], [road for profile_path in profile for road in paths[profile_path]])

        for p in paths.keys():  # すべての経路pについて
            # 車両iが経路pを選択した場合の効用を計算します
            potential_utility = U_i(paths[p], [road for profile_path in profile for road in paths[profile_path]])
            # 現在の経路が他の経路よりも効用が低い場合はナッシュ均衡ではない
            if current_utility < potential_utility:
                is_nash_equilibrium = False
                break
        if not is_nash_equilibrium:
            break

    # ナッシュ均衡であれば、そのプロファイルをナッシュ均衡リストに追加します
    if is_nash_equilibrium:
        nash_equilibria.append(profile)

    # 現在の行動プロファイルに対する全体の目的関数Wを計算します。
    current_W = W([road for p in profile for road in paths[p]])

    # 最大のWを持つプロファイルを更新します。
    if current_W > max_W:
        max_W = current_W
        optimal_profile = profile

# ナッシュ均衡、最適なプロファイル、および最大のWの値を出力します。
for i in nash_equilibria:
    print(i)

print(len(nash_equilibria))
print(optimal_profile, max_W)


('p1', 'p2', 'p3', 'p3')
('p1', 'p3', 'p2', 'p3')
('p1', 'p3', 'p3', 'p2')
('p2', 'p1', 'p3', 'p3')
('p2', 'p3', 'p1', 'p3')
('p2', 'p3', 'p3', 'p1')
('p3', 'p1', 'p2', 'p3')
('p3', 'p1', 'p3', 'p2')
('p3', 'p2', 'p1', 'p3')
('p3', 'p2', 'p3', 'p1')
('p3', 'p3', 'p1', 'p2')
('p3', 'p3', 'p2', 'p1')
12
('p1', 'p1', 'p2', 'p2') 60


,Player 1,Player 2,Player 3,Player 4,Payoff 1,Payoff 2,Payoff 3,Payoff 4
0,p1,p1,p1,p1,7,7,7,7
1,p1,p1,p1,p2,11,11,11,19
2,p1,p1,p1,p3,8,8,8,17
3,p1,p1,p2,p1,11,11,19,11
4,p1,p1,p2,p2,15,15,15,15
...,...,...,...,...,...,...,...,...
76,p3,p3,p2,p2,13,13,9,9
77,p3,p3,p2,p3,9,9,10,9
78,p3,p3,p3,p1,9,9,9,10
79,p3,p3,p3,p2,9,9,9,10
